# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré.

<div class="alert alert-info">
    Pour ce Notebook, on pourra choisir le kernel <code>conda_tensorflow2_p310</code> sur SageMaker.
</div>

In [13]:
!pip install spacy transformers datasets sentencepiece seaborn pandas -q
!mkdir -p data/
!wget -q https://blent-keskia.s3.eu-west-3.amazonaws.com/poc/cyberharcelement/mlma.csv -O data/mlma.csv

In [15]:
!git lfs install
!git clone https://huggingface.co/datasets/Paul/hatecheck-french

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
fatal: destination path 'hatecheck-french' already exists and is not an empty directory.


De même, nous allons utiliser un jeu de données supplémentaire de HuggingFace pour enrichir notre base.

Enfin, spaCy nous permettra de faire des opérations de traitement du texte adapté à la langue française.

In [16]:
!python -m spacy download fr_core_news_sm

2023-03-25 14:19:45.706432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 14:19:47.203228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-25 14:19:47.203402: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-25 14:19:47.203433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [17]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sns.set()

data = pd.read_csv("data/mlma.csv")
hatecheck = pd.read_csv("hatecheck-french/test.csv")

Ce qui est important pour ce POC, c'est de **différencier ce qui relève de la plaisanterie dans un groupe d'amis d'un harcèlement haineux**. Pour cela, la **notion de contexte sera donc très importante**.

Nous allons considérer les sentiments `normal` et `fearful_normal` comme `non-hateful`, de sorte à harmoniser les labels entre les deux jeux de données.

In [18]:
data.loc[data["sentiment"] == "normal", "sentiment"] = "non-hateful"
data.loc[data["sentiment"] == "fearful_normal", "sentiment"] = "non-hateful"

In [70]:
from sklearn.utils import shuffle

# df représente notre DataFrame final du traitement de données
df1 = pd.DataFrame()
df1["Text"] = data["tweet"]
df1["Group"] = data["group"]
df1["Type"] = data["sentiment"]

df2 = pd.DataFrame()
df2["Text"] = hatecheck["test_case"]
df2["Group"] = hatecheck["target_ident"]
df2["Type"] = hatecheck["label_gold"]

df = pd.concat([df1, df2], axis=0)
df = shuffle(df)

Dans notre cas, nous allons rester sur de la classification binaire.

In [71]:
df["Type"].loc[(df1["Type"] != "non-hateful")] = "hateful"
df

,Text,Group,Type
2678,nskslzlzozozozozpeorp brui tah les film star w...,other,hateful
3804,@user mdr t'es vraiment perdu comme mec t'faço...,individual,hateful
582,@user @user flagrant délit de militantisme gau...,left_wing_people,hateful
586,Je vais tirer une balle sur le prochain réfugi...,refugees,hateful
895,@user mediapart islamo - gauchiste ? c est le ...,muslims,non-hateful
...,...,...,...
2751,moix c'est un anti tout pro rien une sorte d'a...,individual,hateful
3322,J'arrive pas à croire cetravelo.,trans people,hateful
931,@user oue ils sont tous attarde miskine,special_needs,hateful
10,#duisburg 4 réfugiés arabes lardent de coups d...,arabs,hateful


Là-aussi pour simplifier l'entraînement du modèle, nous allons fusionner certains groupes.

In [72]:
df["Group"].loc[(df["Group"] == "gay people")] = "gay"
df["Group"].loc[(df["Group"] == "trans people")] = "gay"
df["Group"].loc[(df["Group"] == "black people")] = "african_descent"
df["Group"].loc[(df["Group"] == "Muslims")] = "muslims"
df["Group"].loc[(df["Group"] == "disabled people")] = "special_needs"
df["Group"].loc[(df["Group"] == "refugees")] = "immigrants"
df["Group"].loc[(df["Group"] == "individual")] = "other"

Regardons à nouveau les groupes uniques.

# Préparation des données

Tout d'abord, certaines catégories ne sont pas renseignées : dans ce cas, on considérera alors le groupe d'individus `other`.

In [73]:
df.fillna('other', inplace=True)

Ensuite, puisque notre modèle n'est capable de ne produire d'une classe en sortie en nond eux classes, nous allons créer artificiellement des classes qui combinent le groupe visé ainsi que le sentiment (haineux ou non).

### ➡️ À toi de jouer

Créer la colonne `combined` qui combine à la fois le groupe et le type. Par exemple `other non-hateful` ou `women hateful`.

In [74]:
#df["Combined"] = df["Group"]+" "+df["Type"]
df["Combined"] = df.apply(lambda x: f"{x['Group']} {x['Type']}", axis=1)
df

,Text,Group,Type,Combined
2678,nskslzlzozozozozpeorp brui tah les film star w...,other,hateful,other hateful
3804,@user mdr t'es vraiment perdu comme mec t'faço...,other,hateful,other hateful
582,@user @user flagrant délit de militantisme gau...,left_wing_people,hateful,left_wing_people hateful
586,Je vais tirer une balle sur le prochain réfugi...,immigrants,hateful,immigrants hateful
895,@user mediapart islamo - gauchiste ? c est le ...,muslims,non-hateful,muslims non-hateful
...,...,...,...,...
2751,moix c'est un anti tout pro rien une sorte d'a...,other,hateful,other hateful
3322,J'arrive pas à croire cetravelo.,gay,hateful,gay hateful
931,@user oue ils sont tous attarde miskine,special_needs,hateful,special_needs hateful
10,#duisburg 4 réfugiés arabes lardent de coups d...,arabs,hateful,arabs hateful


Maintenant, nous encodons numériquement les classes : cela permettra au modèle de fournir en sortie une probabilité pour chaque classe.

### ➡️ À toi de jouer

Utiliser un objet `LabelEncoder` de Scikit-Learn pour créer un encodage par dictionnaire et stocker l'encodage de chaque catégorie `combined` dans la colonne `combined_index`.

In [75]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

le = preprocessing.LabelEncoder()

from sklearn.preprocessing import LabelEncoder

# Créer un objet LabelEncoder
le = LabelEncoder()

# Encoder les catégories de la colonne "Combined" en tant que nombres entiers
df['combined_index'] = le.fit_transform(df['Combined'])

# Créer un dictionnaire pour associer chaque catégorie à son index encodé
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))


le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping
df

,Text,Group,Type,Combined,combined_index
2678,nskslzlzozozozozpeorp brui tah les film star w...,other,hateful,other hateful,22
3804,@user mdr t'es vraiment perdu comme mec t'faço...,other,hateful,other hateful,22
582,@user @user flagrant délit de militantisme gau...,left_wing_people,hateful,left_wing_people hateful,18
586,Je vais tirer une balle sur le prochain réfugi...,immigrants,hateful,immigrants hateful,12
895,@user mediapart islamo - gauchiste ? c est le ...,muslims,non-hateful,muslims non-hateful,21
...,...,...,...,...,...
2751,moix c'est un anti tout pro rien une sorte d'a...,other,hateful,other hateful,22
3322,J'arrive pas à croire cetravelo.,gay,hateful,gay hateful,8
931,@user oue ils sont tous attarde miskine,special_needs,hateful,special_needs hateful,24
10,#duisburg 4 réfugiés arabes lardent de coups d...,arabs,hateful,arabs hateful,2


Maintenant, nous allons traiter notre jeu de données car certains mots ne sont pas nécessaires (comme les conjonctions de coordination appelés *stopwords*). Pour cela, nous utilisons la librairie `spacy`.

In [76]:
import spacy

nlp = spacy.load("fr_core_news_sm")

### ➡️ À toi de jouer

Nettoyer la colonne `Text` avec les étapes suivantes.

- Mettre le texte en minuscule et s'assurer qu'il n'y ait pas autre chose que du texte.
- Retirer les métadonnées propres à Twitter comme `@user` ou `@url`.

In [77]:
import regex as re

def clean_text(text):
# Mettre le texte en minuscule et s'assurer qu'il n'y ait pas autre chose que du texte.
    text = text.lower()
    text = re.sub(r"[^a-z ]", "", text)
    
    # Retirer les noms d'utilisateur Twitter comme @user.
    text = re.sub(r"@\S+", "", text)
    
    # Retirer les URL commençant par http ou https.
    text = re.sub(r"https?://\S+", "", text)
    
    return text

# Nettoyer la colonne Text en utilisant la fonction clean_text
df["Text"] = df["Text"].apply(clean_text)
df

,Text,Group,Type,Combined,combined_index
2678,nskslzlzozozozozpeorp brui tah les film star w...,other,hateful,other hateful,22
3804,user mdr tes vraiment perdu comme mec tfaon te...,other,hateful,other hateful,22
582,user user flagrant dlit de militantisme gauchi...,left_wing_people,hateful,left_wing_people hateful,18
586,je vais tirer une balle sur le prochain rfugi ...,immigrants,hateful,immigrants hateful,12
895,user mediapart islamo gauchiste c est le tit...,muslims,non-hateful,muslims non-hateful,21
...,...,...,...,...,...
2751,moix cest un anti tout pro rien une sorte dado...,other,hateful,other hateful,22
3322,jarrive pas croire cetravelo,gay,hateful,gay hateful,8
931,user oue ils sont tous attarde miskine,special_needs,hateful,special_needs hateful,24
10,duisburg rfugis arabes lardent de coups de co...,arabs,hateful,arabs hateful,2


In [79]:
re.sub(r"@\S+", "", "@user @user ben toi aussi te dira de te reconver.")

'  ben toi aussi te dira de te reconver.'

Voyons à quoi notre jeu de données ressemble.

In [78]:
df.head(n=15)

,Text,Group,Type,Combined,combined_index
2678,nskslzlzozozozozpeorp brui tah les film star w...,other,hateful,other hateful,22
3804,user mdr tes vraiment perdu comme mec tfaon te...,other,hateful,other hateful,22
582,user user flagrant dlit de militantisme gauchi...,left_wing_people,hateful,left_wing_people hateful,18
586,je vais tirer une balle sur le prochain rfugi ...,immigrants,hateful,immigrants hateful,12
895,user mediapart islamo gauchiste c est le tit...,muslims,non-hateful,muslims non-hateful,21
2757,apres les rebeus nouveau crdo de fink les enfo...,other,hateful,other hateful,22
3199,jai un seul regret mais je mattarde pas dessus...,other,non-hateful,other non-hateful,23
2533,les personnes qui menvoient des messages du ty...,gay,hateful,gay hateful,8
3211,dans le plus grand des calmes la presse gauchi...,other,hateful,other hateful,22
553,user augmentation des violences proportionnell...,indian/hindu,hateful,indian/hindu hateful,14


Pour terminer, on enregistre le jeu de données nettoyé dans le fichier `data_clean.csv` : c'est lui que l'on utilise **pour entraîner le modèle**.

In [58]:
df.to_csv("data_clean.csv")
df

,Text,Group,Type,Combined,combined_index
2792,si seulement quelquun pouvait touffer ces init...,other,hateful,other hateful,22
3964,user mongol,special_needs,hateful,special_needs hateful,24
2657,dans ce workshop de lesbos des rfugis dcoupent...,other,hateful,other hateful,22
1280,ces musulmans peuvent ressembler des tres hum...,muslims,hateful,muslims hateful,20
1474,ce genre de blacks ne sintgreront jamais on de...,african_descent,hateful,african_descent hateful,0
...,...,...,...,...,...
3567,user user ben toi aussi te dira de te reconver...,other,non-hateful,other non-hateful,23
3700,trop de mongliens dans ce pays,special_needs,hateful,special_needs hateful,24
3547,user user regarde le ce mongol,other,hateful,other hateful,22
89,user cest de la violence dont vous tes le produit,other,non-hateful,other non-hateful,23
